In [9]:
import pygame
import random

# Initialize Pygame
pygame.init()

# Set up the game window
WIDTH = 800
HEIGHT = 600
window = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Space Invaders")

# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

# Player class
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.Surface((50, 40))
        self.image.fill(GREEN)
        self.rect = self.image.get_rect()
        self.rect.centerx = WIDTH // 2
        self.rect.bottom = HEIGHT - 10
        self.speed = 5

    def update(self):
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT] and self.rect.left > 0:
            self.rect.x -= self.speed
        if keys[pygame.K_RIGHT] and self.rect.right < WIDTH:
            self.rect.x += self.speed

    def shoot(self):
        return Bullet(self.rect.centerx, self.rect.top)

# Enemy class
class Enemy(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((40, 40))
        self.image.fill(RED)
        self.rect = self.image.get_rect()
        self.rect.x = x
        self.rect.y = y
        self.speed = 1
        self.direction = 1

    def update(self):
        self.rect.x += self.speed * self.direction

# Bullet class
class Bullet(pygame.sprite.Sprite):
    def __init__(self, x, y):
        super().__init__()
        self.image = pygame.Surface((5, 15))
        self.image.fill(WHITE)
        self.rect = self.image.get_rect()
        self.rect.centerx = x
        self.rect.bottom = y
        self.speed = -10

    def update(self):
        self.rect.y += self.speed
        if self.rect.bottom < 0:
            self.kill()

# Game class
class SpaceInvaders:
    def __init__(self):
        self.clock = pygame.time.Clock()
        self.player = Player()
        self.all_sprites = pygame.sprite.Group(self.player)
        self.enemies = pygame.sprite.Group()
        self.create_enemies()
        self.bullets = pygame.sprite.Group()
        self.font = pygame.font.Font(None, 36)
        self.score = 0

    def create_enemies(self):
        for row in range(5):
            for column in range(11):
                enemy = Enemy(column * 60 + 50, row * 50 + 50)
                self.enemies.add(enemy)
                self.all_sprites.add(enemy)
    
    def run(self):
        running = True
        while running:
            self.clock.tick(60)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                elif event.type == pygame.KEYDOWN:
                    if event.key == pygame.K_SPACE:
                        bullet = self.player.shoot()
                        self.bullets.add(bullet)
                        self.all_sprites.add(bullet)

            self.all_sprites.update()

            # Check if enemies reach the edge of the screen
            for enemy in self.enemies:
                if enemy.rect.right >= WIDTH or enemy.rect.left <= 0:
                    for e in self.enemies:
                        e.rect.y += 20
                        e.direction *= -1
                    break

            # Check for collisions
            hits = pygame.sprite.groupcollide(self.enemies, self.bullets, True, True)
            for hit in hits:
                self.score += 10

            if pygame.sprite.spritecollideany(self.player, self.enemies):
                running = False

            # Draw everything
            window.fill(BLACK)
            self.all_sprites.draw(window)

            # Draw the score
            score_text = self.font.render(f"Score: {self.score}", True, WHITE)
            window.blit(score_text, (10, 10))

            pygame.display.flip()

        pygame.quit()

if __name__ == "__main__":
    game = SpaceInvaders()
    game.run()